In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import openpathsampling as paths
import numpy as np

storage = paths.storage.Storage("mstis.nc", mode="r")

In [2]:
mstis = storage.networks.load(0)

In [3]:
print mstis

Multiple State TIS Network:
RETISTransition: Out B
B -> B or all states except B
Interface: 0.0<opB<0.04
Interface: 0.0<opB<0.09
Interface: 0.0<opB<0.16
RETISTransition: Out C
C -> C or all states except C
Interface: 0.0<opC<0.04
Interface: 0.0<opC<0.09
Interface: 0.0<opC<0.16
RETISTransition: Out A
A -> A or all states except A
Interface: 0.0<opA<0.04
Interface: 0.0<opA<0.09
Interface: 0.0<opA<0.16



In [4]:
stateA = storage.volumes.find_first("A")
stateB = storage.volumes.find_first("B")
stateC = storage.volumes.find_first("C")

In [5]:
print mstis.from_state[stateA]

RETISTransition: Out A
A -> A or all states except A
Interface: 0.0<opA<0.04
Interface: 0.0<opA<0.09
Interface: 0.0<opA<0.16



In [6]:
print mstis.from_state[stateA].ensembles

[<openpathsampling.ensemble.TISEnsemble object at 0x117a3d390>, <openpathsampling.ensemble.TISEnsemble object at 0x117b61490>, <openpathsampling.ensemble.TISEnsemble object at 0x117a3d4d0>]


In [ ]:
mstis.hist_args['max_lambda'] = { 'bin_width' : 0.02, 'bin_range' : (-0.35, 0.35) }
mstis.hist_args['pathlength'] = { 'bin_width' : 5, 'bin_range' : (0, 150) }

In [ ]:
mstis.rate_matrix(storage)

In [ ]:
minus_moves = [d for d in storage.pathmovechanges if mstis.from_state[stateC].movers['minus'][0] in d]
len(minus_moves)

In [ ]:
mstis.movers['minus']

In [ ]:
[mstis.from_state[state].movers['minus'] for state in [stateA, stateB, stateC]]